In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
!cat '/content/drive/MyDrive/fortest/train/HS/13.txt'

@USER @USER @USER يا لطيف.. يا ساتر ..أحمدوا ربكم إنها مستقعدة🌚كيف لو إنها واقفه👌	OFF	NOT_HS


In [ ]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/MyDrive/fortest/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
   
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/MyDrive/fortest/dev",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "/content/drive/MyDrive/fortest/test", batch_size=batch_size
)

print(
    "Number of batches in raw_train_ds: %d"
    % tf.data.experimental.cardinality(raw_train_ds)
)
print(
    "Number of batches in raw_val_ds: %d" % tf.data.experimental.cardinality(raw_val_ds)
)
print(
    "Number of batches in raw_test_ds: %d"
    % tf.data.experimental.cardinality(raw_test_ds)
)

Found 26 files belonging to 2 classes.
Using 21 files for training.
Found 25 files belonging to 2 classes.
Using 5 files for validation.
Found 24 files belonging to 2 classes.
Number of batches in raw_train_ds: 1
Number of batches in raw_val_ds: 1
Number of batches in raw_test_ds: 1


In [ ]:
# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i].decode('utf-8').strip())
        print(label_batch.numpy()[i])
        print('--------------------------------')

@USER اعلامي هلالي وش ترجي منه اي والله الى قال محمد العويس حراق🔥🔥🔥حارق قلبك يا زقان يا المطيًويع من حرق عساه للحرق يا رب جعلني اشوف فيك يوم اسود انت وعائلتك يا ابن الكلب	OFF	NOT_HS
0
--------------------------------
يا التاج ع الراس يا السادة<LF>يا مالك الروح وراعيها❤️	NOT_OFF	NOT_HS
1
--------------------------------
فدوه يا بخت فدوه يا زمن واحد منكم يجيبه	NOT_OFF	NOT_HS
1
--------------------------------
@USER يا بجم يا عجم يا نجم كلنا فالهوا سوا	OFF	NOT_HS
0
--------------------------------
@USER @USER @USER يا لطيف.. يا ساتر ..أحمدوا ربكم إنها مستقعدة🌚كيف لو إنها واقفه👌	OFF	NOT_HS
0
--------------------------------


In [ ]:
import re
import string
import sys
import argparse

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations


arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)





def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)


def custom_standardization(input_data):
    #text = normalize_arabic(input_data)
    text = remove_punctuations(input_data)
    text = remove_diacritics(text)
    text = remove_repeating_char(text)
    return text.decode('utf-8')




# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)


text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)


AttributeError: ignored

In [ ]:
re.sub(r'(.)\1+', r'\1', "haaaaapppppyyy")  

'hapy'

In [ ]:
import string
import re
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    # lowercase = tf.strings.lower(input_data)
    # stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    stripped_html = tf.strings.regex_replace(input_data, "[a-zA-Z]|\d+|[٠١٢٣٤٥٦٧٨٩]", " ")
    stripped_html = tf.strings.regex_replace(stripped_html, "[.،,!?؟\\-”“٪ًَ]", " ")
    
    stripped_html = tf.strings.regex_replace(stripped_html, "[إأآا]", "ا")
    stripped_html = tf.strings.regex_replace(stripped_html, "ة", "ه")
    # stripped_html=tf.strings.regex_replace(stripped_html, "[(\U0001F600-\U0001F92F|\U0001F300-\U0001F5FF|\U0001F680-\U0001F6FF|\U0001F190-\U0001F1FF|\U00002702-\U000027B0|\U0001F926-\U0001FA9F|\u200d|\u2640-\u2642|\u2600-\u2B55|\u23cf|\u23e9|\u231a|\ufe0f)|\u2069|\u2066]+", " ")
    #stripped_html = tf.strings.regex_replace(stripped_html, r'(.)\1+', r'\1')
    stripped_html = tf.strings.regex_replace(stripped_html, r'(.)\1+', r'\1')

    #stripped_html = re.sub(r'(.)\1+', r'\1',str(stripped_html) )
    return tf.strings.regex_replace(stripped_html, "[%s]" % re.escape(string.punctuation), "" )


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

InvalidArgumentError: ignored

In [ ]:
vocab = np.array(vectorize_layer.get_vocabulary())
vocab[:30]

array(['', '[UNK]', 'يا', 'من', 'على', 'ابن', 'و', 'كل', 'فاشل', 'سلمان',
       'انت', 'والله', 'قلبي', 'قال', 'عاش', 'زمالك', 'يوم', 'واحد',
       'ممحون', 'مش', 'لو', 'لسه', 'كرتونه', 'قلبك', 'فين', 'فيك', 'فدوه',
       'رب', 'حيوان', 'حلو'], dtype='<U11')

In [ ]:
encoded_example = vectorize_layer(text_batch)[:20].numpy()
encoded_example

array([[267,  75,  63, ...,   0,   0,   0],
       [  2, 256, 148, ...,   0,   0,   0],
       [ 26,   2, 214, ...,   0,   0,   0],
       ...,
       [  2,  27,   2, ...,   0,   0,   0],
       [ 24,   2, 180, ...,   0,   0,   0],
       [208, 226,   2, ...,   0,   0,   0]])

In [ ]:
for n in range(20):
  print("Original: ", text_batch[n].numpy().decode())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  @USER اعلامي هلالي وش ترجي منه اي والله الى قال محمد العويس حراق🔥🔥🔥حارق قلبك يا زقان يا المطيًويع من حرق عساه للحرق يا رب جعلني اشوف فيك يوم اسود انت وعائلتك يا ابن الكلب	OFF	NOT_HS

Round-trip:  اعلامي هلالي وش ترجي منه اي والله الى قال محمد العويس حراق🔥🔥🔥حارق قلبك يا زقان يا المطي ويع من حرق عساه للحرق يا رب جعلني اشوف فيك يوم اسود انت وعائلتك يا ابن الكلب                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

Original:  يا التاج ع الراس يا السادة<LF>يا مالك الروح وراعيها❤️	NOT_OFF	NOT_HS

Round-trip:  يا التاج ع الراس يا الساده يا مالك الروح وراعيها❤️                 

# Create the model

In [ ]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

history = model.fit(raw_train_ds, epochs=10,
                    validation_data=raw_val_ds, 
                    validation_steps=30,
                    callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)]
                    )


# history=model.fit(train_ds, validation_data=val_ds, epochs=epochs,
#           callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
test_loss, test_acc = model.evaluate(raw_test_ds)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

#epochs_range = range(22)

plt.figure(figsize=(15, 15))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
sample_text = (' يا حبيبي بكل صفاتہ يا دموع و يا خضوع  ')
predictions = model.predict(np.array([sample_text]))
predictions
#NOT hate speech

In [ ]:
sample_text = (' القم يا ايطالي يا ابن الكلب انت و باصك يلعن اهلك  ')
predictions = model.predict(np.array([sample_text]))
predictions
#Hate